In [1]:
from mylib import preprocess
import pandas as pd
import os

pretest = pd.read_csv(os.path.join('data', 'test_users.csv'), header=0, parse_dates=[1,2,3])
pretrain = pd.read_csv(os.path.join('data', 'train_users_2.csv'), header=0, parse_dates=[1,2,3])
df_sessions = pd.read_csv("data\sessions.csv", encoding='utf8')

In [2]:
train, test, y, le = preprocess.make_user_features(pretrain, pretest)

In [3]:
data = pd.concat((train, test), axis=0, ignore_index=True)
final = preprocess.make_sessions_features(data, df_sessions)

In [4]:
final[:train.shape[0]].shape

(213451, 549)

In [5]:
from sklearn.cross_validation import train_test_split
our_train = final[:train.shape[0]]
X_train, X_test, y_train, y_test = train_test_split(our_train, y, test_size=0.33, random_state=42, stratify=y)

In [6]:
X_train.shape

(143012, 549)

In [7]:
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
clf = RandomForestClassifier(n_estimators=160, oob_score=True, n_jobs=-1, criterion='entropy')
clf.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=160, n_jobs=-1,
            oob_score=True, random_state=None, verbose=0, warm_start=False)

In [8]:
import numpy as np
np.count_nonzero(clf.feature_importances_ < 1e-4)

313

In [9]:
unimportant_features = clf.feature_importances_ < 1e-4
clean_X_train = X_train.ix[:, ~unimportant_features]

In [10]:
from sklearn.grid_search import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from pprint import pprint
neigh = KNeighborsClassifier(n_neighbors=2)

param_grid = {"n_neighbors": [1, 2, 4, 8, 16, 32]}

search = GridSearchCV(neigh, param_grid, scoring='f1_weighted', cv=3, verbose=4, n_jobs=3)
search.fit(clean_X_train, y_train)
pprint(sorted(search.grid_scores_, key=lambda x: -x.mean_validation_score))

[Parallel(n_jobs=3)]: Done  19 out of  18 | elapsed: 98.8min remaining:    0.0s
[Parallel(n_jobs=3)]: Done  19 out of  18 | elapsed: 111.7min remaining:    0.0s
[Parallel(n_jobs=3)]: Done  19 out of  18 | elapsed: 117.5min remaining:    0.0s
[Parallel(n_jobs=3)]: Done  19 out of  18 | elapsed: 129.6min remaining:    0.0s
[Parallel(n_jobs=3)]: Done  19 out of  18 | elapsed: 131.2min remaining:    0.0s
[Parallel(n_jobs=3)]: Done  18 out of  18 | elapsed: 131.2min finished


Fitting 3 folds for each of 6 candidates, totalling 18 fits
[mean: 0.54168, std: 0.00111, params: {'n_neighbors': 16},
 mean: 0.54037, std: 0.00059, params: {'n_neighbors': 32},
 mean: 0.53778, std: 0.00158, params: {'n_neighbors': 8},
 mean: 0.52772, std: 0.00106, params: {'n_neighbors': 4},
 mean: 0.49698, std: 0.00090, params: {'n_neighbors': 1},
 mean: 0.49057, std: 0.00084, params: {'n_neighbors': 2}]
